In [8]:
import subprocess
import numpy as np
import pandas as pd
import re
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import redis
from generate_sms_template.generate_template import generate_template, get_cluster_stat

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:67: RuntimeWarning: Error importing matplotlib module. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:67: RuntimeWarning: Error importing matplotlib module. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:67: RuntimeWarning: Error importing matplotlib module. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:40: RuntimeWarning: Error importing draw module, proceeding nevertheless: cairo 1.10.0 is installed; cairo>=1.11.0 is required
  warnings.warn(msg, RuntimeWarning)


In [4]:
import os

In [5]:
os.path

<module 'posixpath' from '/usr/lib/python3.6/posixpath.py'>

In [5]:
redis_client = redis.Redis(host='localhost', port=6379, db=0)

### 原始数据 每个bank.csv

In [39]:
res_root = '/data-0/gaoyu/india_sms/BANK_me_origin_20170901'
path_lis = []
for fpathe,dirs,fs in os.walk(res_root):
    print(fs)
    for f in fs:
        path_lis.append(os.path.join(fpathe,f))
        
path_lis = [i for i in path_lis if i != '/data-0/gaoyu/india_sms/BANK_me_origin_20170901/.DS_Store']
print(path_lis[0])
print(len(path_lis))

['KOTKBK.csv', 'SBICRD.csv', 'BOIIND.csv', 'HDFCBK.csv', 'BKDENA.csv', 'ANDBNK.csv', 'FROMSC.csv', 'MYACCT.csv', 'SCISMS.csv', 'FEDBNK.csv', 'KOTAKB.csv', 'DBSBNK.csv', 'AIRBNK.csv', 'UNIONB.csv', 'ICICIB.csv', 'SBIUPI.csv', 'SARBNK.csv', 'PAYBAK.csv', 'CANBNK.csv', 'BOBTXN.csv', 'IDBIBK.csv', 'KTKBNK.csv', 'INDUSB.csv', 'VIJBNK.csv', 'ZESTMN.csv', 'RBLBNK.csv', 'SYNBNK.csv', 'CORPBK.csv', 'EPFOHO.csv', 'SBIINB.csv', 'RBISAY.csv', 'UMOBIL.csv', 'BNKBZR.csv', 'IDFCBK.csv', 'INDBNK.csv', 'AXISMR.csv', 'SBGMBS.csv', 'PNBSMS.csv', 'SBIOTP.csv', 'CBSSBI.csv', 'CSHBCK.csv', 'PAYTMB.csv', 'MAHABK.csv', 'YESBNK.csv', 'IOBCHN.csv', '.DS_Store', 'ATMSBI.csv', 'HDFCBN.csv', 'AXISBK.csv', 'SBIPSG.csv', 'CITIBK.csv']
/data-0/gaoyu/india_sms/BANK_me_origin_20170901/KOTKBK.csv
50


In [43]:
path_lis[2]

'/data-0/gaoyu/india_sms/BANK_me_origin_20170901/BOIIND.csv'

### 检查0,2 列是不是代表我们要的东西； 先拿10个dispatcher 做图到模版聚类；

In [4]:
bank_name_lis = [i.split('/')[-1].split('.')[0] for i in path_lis]

done_path  = '/data-0/tigergraph/template/'
done_list = os.listdir(done_path)

remain_lis = set(bank_name_lis) - set(done_list)
print('done num:{}, remaining num:{}'.format(len(done_list), len(remain_lis)))

remain_lis = [i for i in path_lis if i.split('/')[-1].split('.')[0] in remain_lis]
z = [print(i) for i in remain_lis]

done num:48, remaining num:2
/data-0/gaoyu/india_sms/BANK_me_origin_20170901/INDUSB.csv
/data-0/gaoyu/india_sms/BANK_me_origin_20170901/SBGMBS.csv


### 跑图聚类

### 每个dispatcher 每个template 的路径

In [19]:
template = '/data-0/tigergraph/template'

template_lis = []
for fpathe,dirs,fs in os.walk(template):
    lis2 = []
    for f in fs:
        if f != 'for_label.txt':
            lis2.append(os.path.join(fpathe,f))
    template_lis.append(lis2)
    
# template_lis[4]

###  每个dispatcher， 对所有的模版按照发生比率排序； 一旦达到85%覆盖率， 停止；输出模版文件名list；

In [20]:
# To_train_path = '/data-0/tigergraph/TO_train/'
dispatcher_thresh_lis = []
for dispatcher in template_lis[1:]:
    idxs = [i.split('/')[-1].split('_')[0] for i in dispatcher]
    nums = [int(i.split('/')[-1].split('_')[1]) for i in dispatcher]
    dic = {}
    ratios = [i/sum(nums) for i in nums]
    for idx, num, ratio in zip(idxs, nums, ratios):
        dic[idx] = [num, ratio]
    
    lis = sorted(dic.items(), key=lambda d: d[1][1], reverse=True)
    percent = 0
    file_names = []
    for i in lis:
        percent += i[1][1]
        if percent < 0.95 and i[1][1] > 0.002:
            name = i[0]+ '_'+ str(i[1][0])
            file_names.append(name)
    path = '/'.join(dispatcher[0].split('/')[:-1])
    final_path = [path + '/' + i for i in file_names]
    dispatcher_thresh_lis.append(final_path)
dispatcher_thresh_lis = [i for i in dispatcher_thresh_lis if i]

### 对所有的模版进行采样， 写到一个总的文件中；

In [21]:
To_train_path = '/data-0/tigergraph/TO_train/'
# Final_Train = '/data-0/tigergraph/Final_train.txt'
Train_bank_samples = []
# with open(Final_Train, 'w+') as ff_train:
for dispatcher in dispatcher_thresh_lis:
    dispatch_name  = dispatcher[0].split('/')[-2]
    train_file = To_train_path + dispatch_name + '.txt'
    # 每一个银行
    with open(train_file, 'w+') as ff:
        for template in dispatcher:
            # 每一个模版
            with open(template, "r") as file:
                tmp1 = file.readlines()
            template_nums = template.split('/')[-1].split('_')[-1]
            template_ids = template.split('/')[-1].split('_')[0]
#                 print(template)
            to_write = tmp1[-1:]
            for text in to_write:
                Train_bank_samples.append((text, template_ids, template_nums, dispatch_name))
#                     ff.write(i)
#                     ff_train.write(i)

template = pd.DataFrame.from_records(Train_bank_samples)

template.columns = ['sms', 'cluster_id', 'cluster_points', 'bank']
template['Cluster_id'] = template.bank + '_' + template.cluster_id
# template = template.drop(['cluster_id', 'bank'],axis=1)
template.head(3)

# res.to_csv('all_train_samples.csv')

,sms,cluster_id,cluster_points,bank,Cluster_id
0,Rs 1500.00 debited from a/c **2140 on 05-10-18...,5,19814,HDFCBK,HDFCBK_5
1,Thank you for using Debit Card ending 2200 for...,0,19785,HDFCBK,HDFCBK_0
2,OTP is 655150 for txn of INR 2000.00 at ONE97 ...,9,17531,HDFCBK,HDFCBK_9


In [22]:
len(template.bank.unique()

SyntaxError: unexpected EOF while parsing (<ipython-input-22-731b09a7edc5>, line 1)

#### 先拿top10 测试


In [23]:
count_bank = template.groupby('bank').count()
top_10_bank = count_bank.sort_values(by ='sms').head(10).index.tolist()

top_10_df = template[template.bank.isin(top_10_bank)]
top_10_df['unique_template_id'] = top_10_df.bank + '_' + top_10_df.cluster_id


/home/tigergraph/test1/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [24]:
top_10_df

,sms,cluster_id,cluster_points,bank,Cluster_id,unique_template_id
551,Open Zero Balance A/c with Kotak Mahindra Bank...,1,1649,CSHBCK,CSHBCK_1,CSHBCK_1
552,Your Kotak Savings Account Here. Open: http://...,0,1473,CSHBCK,CSHBCK_0,CSHBCK_0
553,Your Loan Approved Complete Process: Click ht...,3,387,CSHBCK,CSHBCK_3,CSHBCK_3
632,Open Zero Balance A/c with Kotak Mahindra Bank...,0,2682,MYACCT,MYACCT_0,MYACCT_0
633,Your Kotak Savings Account Here. Open: http:/...,1,1736,MYACCT,MYACCT_1,MYACCT_1
634,Open Zero Balance A/c with Kotak Mahindra Bank...,0,12950,KOTKBK,KOTKBK_0,KOTKBK_0
635,Open Zero Balance A/c with Kotak Mahindra Bank...,1,3690,KOTKBK,KOTKBK_1,KOTKBK_1
636,173075 is your One Time Password for online pu...,0,22968,SBIOTP,SBIOTP_0,SBIOTP_0
692,Dear Customer you have sent Rs.1300.00 via IM...,2,15279,PAYTMB,PAYTMB_2,PAYTMB_2
693,Dear Customer you have sent Rs.429.00 to add-...,1,14887,PAYTMB,PAYTMB_1,PAYTMB_1


### 服务器上预处理 然后 打标签；

### 打完标签变成json， 读取， 与原来的拼接到一起

In [9]:
def labeledDataFromJson(json_path):
    import json
    res=[]
    with open(json_path, 'r') as f:
        for line in f.readlines():
            dic = json.loads(line)
    #         print(dic)
            if dic['annotation'] is not None:
                res.append((dic['content'], dic['annotation']['labels']))
            else:
                res.append((dic['content'], 'error'))

    labeled_df = pd.DataFrame.from_records(res)
    labeled_df.columns = ['sms', 'label']
    return labeled_df

top_10_train_path = '/home/qibo/项目2_sms/综合/bank-top-10-data/sms_top_10_bank.json'
labeled_df = labeledDataFromJson(top_10_train_path)
labeled_df.loc[labeled_df.label=='error','label'] = ['流水转出确认支付密码']

### 基于sms 顺序 可以直接(top_10_template, labeled_top_10_template)拼接上

In [17]:
top_10_df['label'] = labeled_df.label.tolist()

/home/tigergraph/test1/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
labeled_df.iloc[10:12].sms.values

array(['sent_id_10 dear customer you have paid rs. 16. 00 via your paytm bank account xxxxxx1370 at paytm on 13-08-2018 at 18:45:45. updated balance: 36. 03. queries write to us at http://m. p-y. tm/care',
       'sent_id_11 you have received rs. 90. 00 in your savings account xxxxxx9261 via sbi debit card. queries write to us at http://m. p-y. tm/care'],
      dtype=object)

### 测试数据

In [19]:
def get_samples(banklist):
    '''从 banklist 中对每个银行 的每个模版采样N 个样本'''
    To_train_path = '/data-0/tigergraph/TO_train/'
    # Final_Train = '/data-0/tigergraph/Final_train.txt'
    Train_bank_samples = []
    # with open(Final_Train, 'w+') as ff_train:
    for dispatcher in dispatcher_thresh_lis:
        dispatch_name  = dispatcher[0].split('/')[-2]
        train_file = To_train_path + dispatch_name + '.txt'
        # 每一个银行
        with open(train_file, 'w+') as ff:
            for template in dispatcher:
                # 每一个模版
                with open(template, "r") as file:
                    tmp1 = file.readlines()
                template_nums = template.split('/')[-1].split('_')[-1]
                template_ids = template.split('/')[-1].split('_')[0]
                to_write = tmp1[-300:]
                for text in to_write:
                    Train_bank_samples.append((text, template_ids, template_nums, dispatch_name))

    res = pd.DataFrame.from_records(Train_bank_samples)
    res.columns = ['sms', 'cluster_id', 'cluster_points', 'bank']
    res = res[res.bank.isin(banklist)]
    res['unique_tmplate_id'] = res['bank'] + '_' + res['cluster_id'] 
    return res


### 根据拿到的测试数据 来预测该短信类别

In [20]:
from numpy import dot
from numpy.linalg import norm

def get_samples(banklist):
    '''从 banklist 中对每个银行 的每个模版采样N 个样本'''
    To_train_path = '/data-0/tigergraph/TO_train/'
    # Final_Train = '/data-0/tigergraph/Final_train.txt'
    Train_bank_samples = []
    # with open(Final_Train, 'w+') as ff_train:
    for dispatcher in dispatcher_thresh_lis:
        dispatch_name  = dispatcher[0].split('/')[-2]
        train_file = To_train_path + dispatch_name + '.txt'
        # 每一个银行
        with open(train_file, 'w+') as ff:
            for template in dispatcher:
                # 每一个模版
                with open(template, "r") as file:
                    tmp1 = file.readlines()
                template_nums = template.split('/')[-1].split('_')[-1]
                template_ids = template.split('/')[-1].split('_')[0]
                to_write = tmp1[-300:]
                for text in to_write:
                    Train_bank_samples.append((text, template_ids, template_nums, dispatch_name))

    res = pd.DataFrame.from_records(Train_bank_samples)
    res.columns = ['sms', 'cluster_id', 'cluster_points', 'bank']
    res = res[res.bank.isin(banklist)]
    res['unique_tmplate_id'] = res['bank'] + '_' + res['cluster_id'] 
    return res


def preprocess(text, single=False):
    # if text is a df obj:
    if isinstance(text, pd.DataFrame):
        text_ = text.sms.values.tolist()
        final = []
        for idx, single in enumerate(text_):
            if idx % 50000 == 0:
                print('processing sentence num:{}'.format(idx))
            sentence = ''.join(single)
            sentence = sentence.replace('?', ' ')
            sentence = sentence.replace('!', ' ')
            text2 = sentence.replace(',', ' ')
            text3 = text2.replace('.', '. ')
            text4 = [w.lower() for w in text3.split()]
            text4 = [re.search('^rs\.?', s).group(0) + re.sub('[rs.]+',' ',s) 
                     if re.search('^rs\.?[0-9]', s) else s for s in text4]
            final_sent = ' '.join(text4)
            fin = final_sent.join(' \n')
            final.append(fin)
        text['new_sms'] = final
        
    # if text is a string obj:
    elif isinstance(text, str):
        sentence = ''.join(text)
        sentence = sentence.replace('?', ' ')
        sentence = sentence.replace('!', ' ')
        text2 = sentence.replace(',', ' ')
        text3 = text2.replace('.', '. ')
        text4 = [w.lower() for w in text3.split()]
        text4 = [re.search('^rs\.?', s).group(0) + re.sub('[rs.]+',' ',s) 
                 if re.search('^rs\.?[0-9]', s) else s for s in text4]
        final_sent = ' '.join(text4)
        text = final_sent.join(' \n')
        
    else:
        raise Exception('text type not allowed')
    return text
    
def get_cos_similarity(sms, templates):
    '''计算一条新sms 与 每个template 相似度'''
    def cos_sim(a,b):
        return dot(a, b)/(norm(a)*norm(b))
    return [cos_sim(i,sms) for i in templates]

def tfIdfVector(corpus):
    '''corpus is a list of sentences'''
    vectorizer = CountVectorizer()
    transformer = TfidfTransformer()   
    x = vectorizer.fit_transform(corpus)
    tfidf = transformer.fit_transform(x)
    return tfidf.toarray() 

def showClfResult(res, template_df):
    '''展示一条随机采样的短信, 根据与模版短信相似度 --> 得到分类结果。'''
    template_cleaned = preprocess(template_df)
    template_corpus = template_cleaned.new_sms.tolist()
    template_unique_id = template_cleaned.unique_template_id.tolist()
    template_labels = [i[0] if isinstance(i, list) else i for i in template_cleaned.label]
    new_instance_id = None
    if isinstance(res, pd.DataFrame):
        new_instance_ = res.sample()
        new_instance = new_instance_.sms.tolist()
        new_instance = preprocess(new_instance[0], single=True)
        new_instance_id = new_instance_.unique_tmplate_id
        print('new_instance:{}\n此短信来自于模版:{}\n'.format(new_instance, new_instance_id.values[0]))

    elif isinstance(res, str):
        new_instance = preprocess(res, single=True)
        print(new_instance)
    else:
        raise Exception('instance type not allowed')
    template_corpus.append(new_instance)
    
    tfidf = tfIdfVector(template_corpus)
    cos_scores = get_cos_similarity(tfidf[-1], tfidf)[:-1]

    max_score = np.max(cos_scores)
    for i in zip(cos_scores, template_labels, template_unique_id):
        if i[0] == max_score:
            print('元组第三个代表模版ID:{}'.format(i))
            return i[1]

In [21]:
top_10_bank

['SBIUPI',
 'SBIOTP',
 'KOTKBK',
 'MYACCT',
 'BOBTXN',
 'RBISAY',
 'CSHBCK',
 'ATMSBI',
 'UMOBIL',
 'PAYTMB']

In [22]:
top_10_df.head(2)

,sms,cluster_id,cluster_points,bank,unique_template_id,new_sms,label
551,Open Zero Balance A/c with Kotak Mahindra Bank...,1,1649,CSHBCK,CSHBCK_1,open zero balance a/c with kotak mahindra ban...,[其他]
552,Your Kotak Savings Account Here. Open: http://...,0,1473,CSHBCK,CSHBCK_0,your kotak savings account here. open: http:/...,[其他]


In [23]:
new_instances = get_samples(top_10_bank)
showClfResult(new_instances, top_10_df)

processing sentence num:0
new_instance: dear customer your account xxxxxx4067 is credited with 281. 00 for refund. updated balance 603. 04. queries write to us at http://m. p-y. tm/care

此短信来自于模版:PAYTMB_28

元组第三个代表模版ID:(0.6079242253336752, '流水-转入', 'PAYTMB_28')


/home/tigergraph/test1/lib/python3.5/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'流水-转入'

#### 错误识别， 模版补充， 重新训练；

In [77]:
sms_try = 'dear customer you have received rs. 5020. 00 via imps in your account xxxxxxxx5993 from bheemavarapu manoj k (xxxxxx5993). ref no: 823009082098. queries write to us at http://m. p-y. tm/care'

showClfResult(sms_try, top_10_df)


processing sentence num:0
 dear customer you have received rs. 5020. 00 via imps in your account xxxxxxxx5993 from bheemavarapu manoj k (xxxxxx5993). ref no: 823009082098. queries write to us at http://m. p-y. tm/care

元组第三个代表模版ID:(0.49210635006764225, '流水-转出', 'PAYTMB_2')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'流水-转出'

In [82]:
top_10_df.label.astype(str).unique().tolist()

["['其他']",
 "['贷前申请＿审核通过']",
 '流水转出确认支付密码',
 "['流水-转出']",
 "['流水-转入']",
 "['账号异常＿余额不足']",
 "['账号异常＿扣款失败']"]

In [83]:
labels = [i[0] if isinstance(i, list) else i for i in top_10_df.label]
top_10_df.label = labels

In [91]:
top_10_df[top_10_df.label!='其他'].label.unique().tolist()

['贷前申请＿审核通过', '流水转出确认支付密码', '流水-转出', '流水-转入', '账号异常＿余额不足', '账号异常＿扣款失败']

In [2]:
top_10_df

NameError: name 'top_10_df' is not defined

In [ ]:
Rs 2720.00 
date
bal

#### 

NER 添加集

NER 已经训练集

In [98]:
to_add = top_10_df[~top_10_df.label.isin(['其他', '贷前申请＿审核通过'])]

In [101]:
Final_Train_cleaned = '/data-0/tigergraph/NER_bank_to_add.txt'
with open(Final_Train_cleaned, 'w') as fnew:
    text = to_add.sms.values.tolist()
    unique_template_id = to_add.unique_template_id.values.tolist()
    print('file is processing and will be saved in:{}'.format(Final_Train_cleaned))
    for idx, tup in enumerate(zip(text,unique_template_id)):
        single, id_ = tup[0], tup[1]
        if idx % 50000 == 0:
            print('processing sentence num:{}'.format(idx))
        sentence = id_ + ' ' + ''.join(single)
        sentence = sentence.replace('?', ' ')
        sentence = sentence.replace('!', ' ')
        text2 = sentence.replace(',', ' ')
        text3 = text2.replace('.', '. ')
        text4 = [w.lower() for w in text3.split()]
        text4 = [re.search('^rs\.?', s).group(0) + re.sub('[rs.]+',' ',s) 
                 if re.search('^rs\.?[0-9]', s) else s for s in text4]
        final_sent = ' '.join(text4)
        fin = final_sent.join(' \n')
        fnew.write(fin)

file is processing and will be saved in:/data-0/tigergraph/NER_bank_to_add.txt
processing sentence num:0
 sbiotp_0 173075 is your one time password for online purchase of rs 2720. 00 at moto tpsl thru state bank debit card ending 1065. don't share this with anyone

 paytmb_2 dear customer you have sent rs. 1300. 00 via imps from your account xxxxxx1370 to andhra bankaccount xxxxxx4629. ref no: 821621603528. queries write to us at http://m. p-y. tm/care

 paytmb_1 dear customer you have sent rs. 429. 00 to add-money@paytm from your paytm bank savings account xxxxxxxx1370. upi reference: 822919722228. queries write to us at http://m. p-y. tm/care

 paytmb_5 dear customer you have paid rs. 16. 00 via your paytm bank account xxxxxx1370 at paytm on 13-08-2018 at 18:45:45. updated balance: 36. 03. queries write to us at http://m. p-y. tm/care

 paytmb_4 you have received rs. 90. 00 in your savings account xxxxxx9261 via sbi debit card. queries write to us at http://m. p-y. tm/care

 paytmb_3

### 看到没， 每个句子前面有 cluster id, 在测试的时候也可以添加上！

In [ ]:
a/c 
Rs 1500.00
05-10-18
bal: 
balance
curr o/s
Card ending 2200
creditcard 6950

credited to
credited with
deposited in
debited from
debited for
withdrawn
transfer

registration
otp

In [131]:
temp[30:50]

[' sent_id_30 verification code for your application is 596154. this code is only for your identity verification purpose.\n',
 ' sent_id_31 dear hdfcbank cardmember payment of rs 21350 was credited to your card ending 6367 on 08/sep/2018.\n',
 ' sent_id_32 take control of your bill payments with hdfc bank billpay. register & pay bills for 250+ billers automatically and get rewarded too. visit: login to netbanking>billpay&recharge. click: hdfcbk. net/o3w4v6m\n',
 ' sent_id_33 363776 is secret one time password (otp) for netbanking third party fund transfer registration. do not share it with anyone.\n',
 ' sent_id_34 041926 is your secret otp to add payee remani vijayan a/c no ending in 408 for fundstransfer. ref no. -xxxx9184. do not share it with anyone.\n',
 ' sent_id_35 dear customer you have done 6 txns on non hdfc bank non metro atm till 7:30 pm on 31 jul 2018. check free txns eligibility on www. hdfcbank. com\n',
 ' sent_id_36 otp is 754047 for ivr/online trxn on your hdfc bank ca

### 下午看看 gt 原理

In [ ]:
    g = gt.Graph()
    for file in glob.glob('{}/*'.format(group_dir)):
        if file.split('_')[-1] != '0':
            df = pd.read_csv(file, header=None)
            g.add_edge_list(df[[0, 1]].values)

    print("n_vertices in graph: {}".format(g.num_vertices()))

    print("finding connect components...")
    comp, hist = gt.label_components(g, directed=False)
    hist_dict = dict(enumerate(hist))